当前主流的大模型都是 decoder-base 的模型结构，分为 prefill 和 decode 两个阶段。其中 decode 时，每次在生成一个 token 时候，都要频繁跟访存交互，加载 KV-Cache，再通过多层网络做完整的前向计算。对于这样的访存密集型的任务，通常会因为访存效率形成训练或推理的瓶颈。

MTP 方法的核心思想是，通过解码阶段的优化，将 1-token 的生成，转变成 multi-token 的生成，从而提升训练和推理的性能。具体来说，在训练阶段，一次生成多个后续 token，可以一次学习多个位置的 label，进而有效提升样本的利用效率，提升训练速度；在推理阶段通过一次生成多个 token，实现成倍的推理加速来提升推理性能。

也就是说，MTP 是从推理阶段的效率角度出发得到的一种解决方案；我们观察到其在形式上与 World Model 的对偶性。因为很自然的，在纯文本模态中的推理，其状态空间和动作空间的关系是
$$s_t = (a_{1}, a_{2}, ..., a_{t-1})$$
假若 MTP 每次多预测的头数是 $k$，则在当前时刻 $t$ 预测之后的 $a_{t}, a_{t+1}, ..., a_{t+k}$，且 $s_t$ 已知，自然也就等同于预测
$$s_{t+i} = s_t \circ (a_{t}, a_{t+1}, ..., a_{t+i}), \quad \text{s.t. } i \in [1, k]$$